# Sentiment Analysis of Description Using Google API

In [1]:
import numpy as np 
import pandas as pd 
import pickle 

Extracting the sentiment score and magnitude from the json sentiment file for the first id

In [3]:
pickle_in = open("cleanedData/train_cleaned_Features_v2.pk","rb")
train = pickle.load(pickle_in)
sample_id = train['PetID'][0]
print(sample_id)

86e1089a3


In [4]:
#read json file for one pet
import json
from pprint import pprint

with open("train_sentiment/{}.json".format(sample_id), "r") as f:
    sample = json.load(f)
print(sample)

{'sentences': [{'text': {'content': 'Nibble is a 3+ month old ball of cuteness.', 'beginOffset': -1}, 'sentiment': {'magnitude': 0.8, 'score': 0.8}}, {'text': {'content': 'He is energetic and playful.', 'beginOffset': -1}, 'sentiment': {'magnitude': 0.8, 'score': 0.8}}, {'text': {'content': 'I rescued a couple of cats a few months ago but could not get them neutered in time as the clinic was fully scheduled.', 'beginOffset': -1}, 'sentiment': {'magnitude': 0.2, 'score': -0.2}}, {'text': {'content': 'The result was this little kitty.', 'beginOffset': -1}, 'sentiment': {'magnitude': 0, 'score': 0}}, {'text': {'content': 'I do not have enough space and funds to care for more cats in my household.', 'beginOffset': -1}, 'sentiment': {'magnitude': 0.1, 'score': 0.1}}, {'text': {'content': "Looking for responsible people to take over Nibble's care.", 'beginOffset': -1}, 'sentiment': {'magnitude': 0.3, 'score': 0.3}}], 'tokens': [], 'entities': [{'name': 'Nibble', 'type': 'ORGANIZATION', 'meta

From this file, we can see that each sentence has been given a sentiment score and magnitude and also the whole description as a whole has been given a documentSentiment score and magnitude index which we will now extract for analysis

In [7]:
print(sample['documentSentiment']['magnitude'])

2.4


Creating a file for all the sentiments and then extracting the PetIDs from them rather than using the PetIDs from the train set to match to the sentiments because not all petIDs in the train set has a sentiment score and magnitude.

In [8]:
import glob
train_sentiment_files = sorted(glob.glob('train_sentiment/*.json'))
train_df_ids = train[['PetID']]
train_df_sentiment = pd.DataFrame(train_sentiment_files)
train_df_sentiment.columns = ['sentiment_filename']
train_sentiment_pets = train_df_sentiment['sentiment_filename'].apply(lambda x: x.split('/')[-1].split('.')[0])
train_df_sentiment = train_df_sentiment.assign(PetID=train_sentiment_pets)
print(len(train_sentiment_pets.unique()))


14442


In [9]:
pets = train_sentiment_pets.apply(lambda x: x.split("\\")[1])

In [10]:
#create dataframe and then extract sentiment score and sentiment magnitude for each petID
dataset = pd.DataFrame()

#remove pets that dont have sentiment data
dataset['PetID'] = pets

In [11]:
dataset['PetID'].head()

0    0008c5398
1    000a290e4
2    000fb9572
3    0011d7c25
4    00156db4a
Name: PetID, dtype: object

In [12]:
train_df_sentiment.sentiment_filename.head()

0    train_sentiment\0008c5398.json
1    train_sentiment\000a290e4.json
2    train_sentiment\000fb9572.json
3    train_sentiment\0011d7c25.json
4    train_sentiment\00156db4a.json
Name: sentiment_filename, dtype: object

Extracting all the sentiment scores and magnitudes for all available petIDs 

In [13]:
for i  in train_df_sentiment.sentiment_filename:
    with open(i, "r", errors='ignore') as f:
        sample = json.load(f)
        pets = i.split("\\")[1].split(".")[0]
        dataset.loc[dataset['PetID']==pets,'Magnitude'] = sample['documentSentiment']['magnitude']
        dataset.loc[dataset['PetID']==pets,'Score'] = sample['documentSentiment']['score']

dataset.head()

,PetID,Magnitude,Score
0,0008c5398,2.8,0.7
1,000a290e4,0.6,0.3
2,000fb9572,0.8,0.3
3,0011d7c25,0.8,0.8
4,00156db4a,1.8,0.0


In [45]:
#dataset.to_csv('sentiments.csv', index=False)
pickle_out = open("sentiment/sentiments.pickle","wb")
pickle.dump(dataset, pickle_out)
pickle_out.close()

# Sentiment Analysis Using Opinion based Feature Mining

In [15]:
from nltk.stem import *
from nltk import pos_tag
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
ps = SnowballStemmer('english')
import re
    
#from stemming.porter2 import stem
from nltk.corpus import stopwords

THRESHOLD = 200

In [16]:
train.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Description,PetID,PhotoAmt,AdoptionSpeed,NumColors,IsMixedBreed,NameFrequency,Magnitude,Score,WordCount
0,2,Nibble,3,299,0,1,1,7,0,1,...,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,2,0,1,2.4,0.3,69
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,2,0,22,0.7,-0.2,23
2,1,Brisco,1,307,0,1,2,7,0,2,...,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3,2,0,1,3.7,0.2,69
3,1,Miko,4,307,0,2,1,2,0,2,...,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2,2,0,11,0.9,0.9,25
4,1,Hunter,1,307,0,1,1,0,0,2,...,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2,1,0,5,3.7,0.6,81


In [17]:
df_train = pd.DataFrame()
df_train['PetID'] = train['PetID']
df_train['Description'] = train['Description']

In [18]:
df_train.head()

,PetID,Description
0,86e1089a3,Nibble is a 3+ month old ball of cuteness. He ...
1,6296e909a,I just found it alone yesterday near my apartm...
2,3422e4906,Their pregnant mother was dumped by her irresp...
3,5842f1ff5,"Good guard dog, very alert, active, obedience ..."
4,850a43f90,This handsome yet cute boy is up for adoption....


In [19]:
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tobel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Tobel\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tobel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

importing the package we used for the program including pos_tag and stem algorithm.

In [20]:
noun_list = []

initializing the list for nouns we select for the features

# Data preprocessing & High Adjective Count (HAC) Algorithm(modified)

In [21]:
nouns_score={}
df = df_train
for index, row in df.iterrows():
    #process the text data
    #str(dataset['text'][i])
    row[1] = re.sub('[^A-Za-z0-9\s]+', '', str(row[1])).lower()
    #tokenize the sentence into words
    text = word_tokenize(row[1])
    #pos_tag the word of the text
    post_tag=nltk.pos_tag(text)
    length=len(post_tag)
    for i in range (length):
        if i==length-1:
            continue
        if post_tag[i][1]=='JJ':
            important=i
            count=1
            while(1):
                #print 'a'
                #print i
                #count=1
                if count%2!=0:
                    count=count*1
                else:
                    count=count*(-1)
                #print str(count)+' this is count'
                important = important+count
                #print important
                #print important
                if post_tag[important][1]=='NN':
                    if post_tag[important][0] in nouns_score:
                        #print 'b'
                        nouns_score[post_tag[important][0]]=nouns_score[post_tag[important][0]]+1
                        break
                    else:
                        #print 'd'
                        nouns_score[post_tag[important][0]]=1
                        break
                if important == 0 or important == length-1:
                    #print 'c'
                    break
                count=abs(count)+1
print("done")

done


Steps
1. Preprocessing the text data
2. Tokenize the sentence/paraphrase
3. Apply POStag to the sentence/paraphrase
4. for each adjective in the sentence/paraphrase, find the closest noun and add them into nouns_score which is a dictionary

In [22]:
#check the stopwords and then delete the element from the dictionary
stoplist_tw=['time','week','today','night','year','day','Year','morning','weekend','timing','hour','day',
            'way','dont','end']

stopwordsss=[]
for key in nouns_score:
    if key in stopwords.words('english') or key in stoplist_tw or len(key)==1 or len(key)==2:
        stopwordsss.append(key)
for i in range(len(stopwordsss)):
    del nouns_score[stopwordsss[i]]

Removing stop words from the list

In [23]:
stem_word_list={}
final_result={}
ultimate_result={}
for key in nouns_score:
    #print key
    #print stem(key)
    if ps.stem(key) in stem_word_list.keys():
        stem_word_list[ps.stem(key)].append(key)
    else:
        stem_word_list[ps.stem(key)]=[key]
    if ps.stem(key) in final_result:
        final_result[ps.stem(key)]=final_result[ps.stem(key)]+nouns_score[key]
    else:
        final_result[ps.stem(key)]=nouns_score[key] 
        
for key in final_result:
    if key in stem_word_list.keys():
        ultimate_result[tuple(stem_word_list[key])]=final_result[key]
        
print(ultimate_result)

{('care', 'caring', 'careful'): 1064, ('ball',): 32, ('couple',): 65, ('kitty', 'kittys', 'kittie'): 262, ('space',): 194, ('household',): 42, ('home',): 4425, ('mother',): 428, ('owner',): 1463, ('jaya',): 69, ('roadside',): 55, ('tying', 'tie'): 97, ('pls',): 364, ('guard',): 323, ('obedience', 'obedient'): 124, ('master',): 40, ('boy',): 693, ('weve',): 13, ('age',): 502, ('brat',): 3, ('kitten',): 903, ('pak',): 2, ('sex',): 28, ('feisty', 'feistiness'): 4, ('cat', 'cats'): 2624, ('pitch',): 3, ('adopter', 'adoption', 'adopt'): 2886, ('playful', 'playing', 'play', 'playfulness', 'played'): 563, ('minute',): 10, ('nap',): 4, ('house', 'housing'): 824, ('vaccination', 'vaccinate', 'vaccine', 'vaccin'): 536, ('month',): 630, ('forever',): 144, ('please',): 2057, ('puppy', 'puppys', 'puppie'): 461, ('jenny',): 13, ('posting', 'post'): 26, ('hairevident',): 2, ('batch',): 23, ('bandar',): 44, ('condition',): 375, ('food',): 1240, ('belt',): 1, ('neighbourhood',): 39, ('neighbour', 'neig

Apply Stem to the noun_score. Then add the count of the words which has same stem together. 
This is what I changed to HAC algorithm. Since if I apply stemming before POStag. The stem of some adjective words may change to noun. Then it will affect our reult.

In [24]:
result=[]
for key in ultimate_result:
    if ultimate_result[key]>200:
        result.append(key)
print(result)
print("done")

[('care', 'caring', 'careful'), ('kitty', 'kittys', 'kittie'), ('home',), ('mother',), ('owner',), ('pls',), ('guard',), ('boy',), ('age',), ('kitten',), ('cat', 'cats'), ('adopter', 'adoption', 'adopt'), ('playful', 'playing', 'play', 'playfulness', 'played'), ('house', 'housing'), ('vaccination', 'vaccinate', 'vaccine', 'vaccin'), ('month',), ('please',), ('puppy', 'puppys', 'puppie'), ('condition',), ('food',), ('kindness', 'kind'), ('anyone',), ('cute', 'cuteness'), ('female',), ('area',), ('hair', 'hairs'), ('loving', 'love'), ('fur', 'furs'), ('toilet',), ('face',), ('body',), ('attention', 'attentive'), ('caging', 'cage'), ('place',), ('family', 'familys'), ('companion',), ('dog', 'dogs'), ('litter',), ('baby', 'babys'), ('heart', 'heartful'), ('mix', 'mixed'), ('bit',), ('pup', 'pups'), ('life',), ('temperament',), ('coat', 'coating'), ('size',), ('personality', 'person', 'personally'), ('health',), ('vet',), ('male', 'malee'), ('pet', 'petting', 'pets'), ('contact',), ('commit

In the end, we compare the count of each noun with the THRESHOLD we set to build a list with only high-frequency words.

In [25]:
feature=[]
for i in range (len (result)):
    if len(result[i])!=1:
        for j in range(len(result[i])):
            feature.append(result[i][j])
    else:
        feature.append(result[i][0])
print(feature)

['care', 'caring', 'careful', 'kitty', 'kittys', 'kittie', 'home', 'mother', 'owner', 'pls', 'guard', 'boy', 'age', 'kitten', 'cat', 'cats', 'adopter', 'adoption', 'adopt', 'playful', 'playing', 'play', 'playfulness', 'played', 'house', 'housing', 'vaccination', 'vaccinate', 'vaccine', 'vaccin', 'month', 'please', 'puppy', 'puppys', 'puppie', 'condition', 'food', 'kindness', 'kind', 'anyone', 'cute', 'cuteness', 'female', 'area', 'hair', 'hairs', 'loving', 'love', 'fur', 'furs', 'toilet', 'face', 'body', 'attention', 'attentive', 'caging', 'cage', 'place', 'family', 'familys', 'companion', 'dog', 'dogs', 'litter', 'baby', 'babys', 'heart', 'heartful', 'mix', 'mixed', 'bit', 'pup', 'pups', 'life', 'temperament', 'coat', 'coating', 'size', 'personality', 'person', 'personally', 'health', 'vet', 'male', 'malee', 'pet', 'petting', 'pets', 'contact', 'commitment', 'commit', 'breed', 'breeding', 'environment', 'lover', 'someone', 'birth', 'tail', 'fee', 'girl', 'call', 'coloring', 'color', '

Using the result we get from HAC. Put all of words which in result list into feature list

# Using RandomClassifer 

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
tokenizer = TweetTokenizer()
vectorizer = TfidfVectorizer(ngram_range=(1, 2), tokenizer=tokenizer.tokenize)


C:\Users\Tobel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Importing the tokenizer to vectorize the dataset 

In [27]:
#vectorize the whole data
#Baseline model
tfidf_vec = TfidfVectorizer(stop_words='english',ngram_range=(1,2),encoding='utf-8')
tfidf_vec.fit_transform(train['Description'].values.astype("U").tolist())
X = tfidf_vec.transform(train['Description'].astype("U").values.tolist())

In [28]:
#evaluating model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
y = train['AdoptionSpeed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Fitting RandomClassifier to fit the model

In [29]:
rf = RandomForestClassifier(max_depth=77,n_estimators=250,min_samples_split=2,min_samples_leaf=1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=77, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Using model to predict

In [30]:
pred = rf.predict(X_test)

Using RSME and Accracy score to evaluate the model

In [31]:
#using rmse to evaluate model
rmse = mean_squared_error(y_test, pred)
rmse

2.449070331447049

In [32]:
#accuracy score using only description column
acc = accuracy_score(y_test, pred)
acc

0.3728779304769604

# Using TFIDF to get the weights of features

In [33]:
import eli5
for i in range(5):
    print(f'Example of Adoption speed {i}')
    text = train.loc[train['AdoptionSpeed'] == i, 'Description'].values[0]
    print(text)
    display(eli5.show_prediction(rf, doc=text, vec=tfidf_vec, top=5))

Example of Adoption speed 0
I just found it alone yesterday near my apartment. It was shaking so I had to bring it home to provide temporary care.


Example of Adoption speed 1
anyone within the area of ipoh or taiping who interested to adopt my cat can contact my father at this number (mazuvil)or can just email me. currently bulat is at my hometown at perak but anyone outside the area still want to adopt can travel there to my hometown.there is a lot of cats in my house rite now..i think i should let one of them go to a better owner who can give better attention to him.


Example of Adoption speed 2
Nibble is a 3+ month old ball of cuteness. He is energetic and playful. I rescued a couple of cats a few months ago but could not get them neutered in time as the clinic was fully scheduled. The result was this little kitty. I do not have enough space and funds to care for more cats in my household. Looking for responsible people to take over Nibble's care.


Example of Adoption speed 3
Their pregnant mother was dumped by her irresponsible owner at the roadside near some shops in Subang Jaya. Gave birth to them at the roadside. They are all healthy and adorable puppies. Already dewormed, vaccinated and ready to go to a home. No tying or caging for long hours as guard dogs. However, it is acceptable to cage or tie for precautionary purposes. Interested to adopt pls call me.


Example of Adoption speed 4
Very manja and gentle stray cat found, we would really like to find a home for it because we cannot keep her for ourselves for long. Has a very cute high pitch but soft meow. Please contact me if you would be interested in adopting.


Using ELI5 to display the weights of our main features explained by the RandomForests model

In [34]:
eli5.show_weights(rf, vec=tfidf_vec, top=50, feature_filter=lambda x: x in feature)

Weight,Feature
0.0042 ± 0.0072,home
0.0035 ± 0.0082,adoption
0.0032 ± 0.0058,adopt
0.0031 ± 0.0056,contact
0.0030 ± 0.0085,kitten
0.0030 ± 0.0073,dog
0.0029 ± 0.0052,puppy
0.0029 ± 0.0071,pls
0.0022 ± 0.0039,playful
0.0020 ± 0.0039,cute


These are the main features that pet adopters care more about and will definitely have an influence in adopting a pet. The top 10 most influential features selected are: home, contact, pls, playful, cute, commitment, loving, litter, care, mother. Words such as 'cat','dog', 'adoption' are expected to show up in the description so will not carry any distinctive importance.

# Finding descriptions which have the 'features' in them and creating dummy variables for the features

In [35]:
import re

def split_it(x):
    x = str(x)
    return re.findall('(home|contact|pls|playful|cute|commitment|loving|litter|care|mother)', x)

df_train['test'] = df_train['Description'].apply(lambda x: split_it(x))
#df['Season2'] = df['Season'].apply(lambda x: split_it(x))
#if word in list exists in column then add it in column

In [36]:
df_train.head()

,PetID,Description,test
0,86e1089a3,nibble is a 3 month old ball of cuteness he is...,"[cute, playful, care, care]"
1,6296e909a,i just found it alone yesterday near my apartm...,"[home, care]"
2,3422e4906,their pregnant mother was dumped by her irresp...,"[mother, home, pls]"
3,5842f1ff5,good guard dog very alert active obedience wai...,[]
4,850a43f90,this handsome yet cute boy is up for adoption ...,"[cute, playful, cute, loving, home, care]"


In [37]:
dummy = pd.get_dummies(df_train.test.apply(pd.Series).stack()).sum(level=0)

In [38]:
dummy.head()

,care,commitment,contact,cute,home,litter,loving,mother,playful,pls
0,2,0,0,1,0,0,0,0,1,0
1,1,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,1,0,1
4,1,0,0,2,1,0,1,0,1,0
6,0,0,1,0,2,0,0,0,0,0


In [39]:
df_train = pd.concat([df_train,dummy],axis=1)

In [40]:
df_train.head()

,PetID,Description,test,care,commitment,contact,cute,home,litter,loving,mother,playful,pls
0,86e1089a3,nibble is a 3 month old ball of cuteness he is...,"[cute, playful, care, care]",2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,6296e909a,i just found it alone yesterday near my apartm...,"[home, care]",1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3422e4906,their pregnant mother was dumped by her irresp...,"[mother, home, pls]",0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,5842f1ff5,good guard dog very alert active obedience wai...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,850a43f90,this handsome yet cute boy is up for adoption ...,"[cute, playful, cute, loving, home, care]",1.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0


In [41]:
df_train = df_train.drop(['test'],axis=1)

In [42]:
res = df_train.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna(''))
res.head()

,PetID,Description,care,commitment,contact,cute,home,litter,loving,mother,playful,pls
0,86e1089a3,nibble is a 3 month old ball of cuteness he is...,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,6296e909a,i just found it alone yesterday near my apartm...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3422e4906,their pregnant mother was dumped by her irresp...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,5842f1ff5,good guard dog very alert active obedience wai...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,850a43f90,this handsome yet cute boy is up for adoption ...,1.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0


In [43]:
res = res.drop(['Description'],axis=1)

In [44]:
#exporting
#res.to_csv('sentiment_features.csv')
pickle_out = open("sentiment/sentiment_features.pickle","wb")
pickle.dump(res, pickle_out)
pickle_out.close()